最近在看《机器学习实战》，里面有一个朴素贝叶斯理论，这个理论并不难，不过书上说得挺绕，网上有个博客挺好，简单明了，可以看一下：[朴素贝叶斯分类器的应用](http://www.ruanyifeng.com/blog/2013/12/naive_bayes_classifier.html)。通过病人分类和账号分类的例子，应该就比较清楚朴素贝叶斯分类器是干啥的了，算出在哪个特征（Feature）的前提下，分类（Category）的概率最高，就划分到那个分类中就行了。我们使用书上狗狗论坛的例子，有如下几段文字，将其分类为侮辱性和非侮辱性，0代表正常言论，1代表侮辱言论：

In [8]:
import numpy as np

def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]    #1 is abusive, 0 not
    return postingList,classVec

In [9]:
listOPosts,listClasses = loadDataSet()
print(listOPosts)
print(listClasses)

[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'], ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'], ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'], ['stop', 'posting', 'stupid', 'worthless', 'garbage'], ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'], ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
[0, 1, 0, 1, 0, 1]


接下来就是统计有哪些单词，这些单词将会组成我们的特征（Feature）

In [10]:
def createVocabList(dataSet):
    vocabSet = set([])  #create empty set
    for document in dataSet:
        vocabSet = vocabSet | set(document) #union of the two sets
    return list(vocabSet)

In [11]:
myVocabList = createVocabList(listOPosts)
print(myVocabList)

['help', 'maybe', 'how', 'steak', 'stupid', 'problems', 'I', 'dalmation', 'my', 'ate', 'mr', 'is', 'not', 'so', 'flea', 'park', 'cute', 'to', 'stop', 'worthless', 'posting', 'licks', 'dog', 'take', 'love', 'please', 'garbage', 'quit', 'buying', 'food', 'has', 'him']


在书中，使用了词向量这一概念，这样做的好处是一切向量化，好处理，不过也不太适合初学者理解，所以我的博客中想尝试不用词向量，这样看起来更直观。现在开始统计每个单词的在侮辱性和非侮辱性语句中出现的概率也就是统计$P(f_{i}|c=0)$和$P(f_{i}|c=1)$，这里的$f_{i}$指的是某个单词

In [76]:
def getProb(postingList,classVec):
    classSet=set(classVec)#统计有多少种类别
    classProb=dict((x,classVec.count(x)/len(classVec)) for x in  classVec)
    feature=[]
    for c in classSet:
        feature.append([y for x,i in zip(postingList,classVec) if i==c for y in x])
    allCount=[len(x) for x in feature]
    prob=[]
    for w,c in zip(feature,allCount):
        prob.append(dict((x, w.count(x)/c) for x in w))#统计侮辱性语句中每个单词出现的数目
    return [prob,classProb]

In [77]:
getProb(listOPosts,listClasses)

[[{'I': 0.041666666666666664,
   'ate': 0.041666666666666664,
   'cute': 0.041666666666666664,
   'dalmation': 0.041666666666666664,
   'dog': 0.041666666666666664,
   'flea': 0.041666666666666664,
   'has': 0.041666666666666664,
   'help': 0.041666666666666664,
   'him': 0.08333333333333333,
   'how': 0.041666666666666664,
   'is': 0.041666666666666664,
   'licks': 0.041666666666666664,
   'love': 0.041666666666666664,
   'mr': 0.041666666666666664,
   'my': 0.125,
   'please': 0.041666666666666664,
   'problems': 0.041666666666666664,
   'so': 0.041666666666666664,
   'steak': 0.041666666666666664,
   'stop': 0.041666666666666664,
   'to': 0.041666666666666664},
  {'buying': 0.05263157894736842,
   'dog': 0.10526315789473684,
   'food': 0.05263157894736842,
   'garbage': 0.05263157894736842,
   'him': 0.05263157894736842,
   'maybe': 0.05263157894736842,
   'not': 0.05263157894736842,
   'park': 0.05263157894736842,
   'posting': 0.05263157894736842,
   'quit': 0.05263157894736842,
 

这样我们就知道了所有的单词在某一类别的概率了，为了让大家更直观，我们这里手动算1个：  
$P(dog|c=0)$的计算：dog在正常语句中出现了1次，正常语句一共有24个（重复单词也重复计算），所以$P(dog|c=0)=\frac{1}{24}=0.0417$  
$P(dog|c=1)$的计算：dog在侮辱语句中出现了2次，正常语句一共有19个，所以$P(dog|c=0)=\frac{2}{19}=0.1052$